Створити віртуальне середовище (venv) в якому будуть встановлені всі необхідні бібліотеки та налаштування для даної лабораторної роботи

Середовище я створив вручну, користуючись візуальним інтерфейсом Anaconda Navigator

In [13]:
!conda install pandas numpy ipython matplotlib urllib3 -y
print("Requirements satisfied")
!conda info --envs
print("* - current venv")

Channels:
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.

Requirements satisfied

# conda environments:
#
base                   D:\Users\Bogdan\anaconda3
python_3-8           * D:\Users\Bogdan\anaconda3\envs\python_3-8

* - current venv


Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження. 
Передбачити повторні запуски скрипту, довантаження нових даних та колізію даних; 


In [77]:
import urllib.request
import os
from datetime import datetime
import hashlib

def calculate_file_hash(file_path):
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

def download_data(region_ID):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={region_ID}&year1=1981&year2=2024&type=Mean"
    current_datetime = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    csv_file_path = f'{dir_for_data}/NOAA_ID_{region_ID}_{current_datetime}.csv'
    
    try:
        vhi_url = urllib.request.urlopen(url)
        # Записуємо нові дані у файл
        with open(csv_file_path, 'wb') as new_file:
            new_file.write(vhi_url.read())

        # Обчислюємо хеш нового файлу
        new_file_hash = calculate_file_hash(csv_file_path)

        # Перевіряємо, чи існує csv-файл з ідентичним вмістом
        region_file_found = False
        for existing_file in os.listdir(dir_for_data):
            existing_file_path = f"{dir_for_data}/{existing_file}"

            if os.path.isfile(existing_file_path):
                if existing_file_path != csv_file_path:
                    existing_file_hash = calculate_file_hash(existing_file_path)

                    if existing_file_hash == new_file_hash:
                        print(f"VHI for region {region_ID} is already up to date in {existing_file_path}")
                        os.remove(csv_file_path)  # Видаляємо новий файл, якщо знайдений файл з ідентичним хешем
                        region_file_found = True
                        break

        # Якщо дані нові, створюємо новий файл
        if not region_file_found:
            print(f"Created csv-file for VHI in path: {csv_file_path}")
                
    except Exception as e:
        print(f"Error downloading VHI for region {region_ID}: {e}")

dir_for_data = "VHI_data"
if not os.path.exists(dir_for_data):
    os.makedirs(dir_for_data)

# Завантажуємо дані кожної адмін. одиниці України, з індексом 1-27
for i in range(1, 28):
    download_data(i)

VHI for region 1 is already up to date in VHI_data/NOAA_ID_1_2025-03-06_20-10-11.csv
VHI for region 2 is already up to date in VHI_data/NOAA_ID_2_2025-03-06_20-10-13.csv
VHI for region 3 is already up to date in VHI_data/NOAA_ID_3_2025-03-06_20-10-14.csv
VHI for region 4 is already up to date in VHI_data/NOAA_ID_4_2025-03-06_20-10-16.csv
VHI for region 5 is already up to date in VHI_data/NOAA_ID_5_2025-03-06_20-10-17.csv
VHI for region 6 is already up to date in VHI_data/NOAA_ID_6_2025-03-06_20-10-19.csv
VHI for region 7 is already up to date in VHI_data/NOAA_ID_7_2025-03-06_20-10-20.csv
VHI for region 8 is already up to date in VHI_data/NOAA_ID_8_2025-03-06_20-10-22.csv
VHI for region 9 is already up to date in VHI_data/NOAA_ID_9_2025-03-06_20-10-24.csv
Created csv-file for VHI in path: VHI_data/NOAA_ID_10_2025-03-06_21-50-33.csv
VHI for region 11 is already up to date in VHI_data/NOAA_ID_11_2025-03-06_20-10-27.csv
VHI for region 12 is already up to date in VHI_data/NOAA_ID_12_2025-03

Зчитати завантажені текстові файли у фрейм. Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [133]:
import os
import pandas as pd

def load_data_from_directory(dir_for_data):
    # Імена стовпців повинні бути зрозумілими та без спеціальних символів
    column_names = ["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI", "Region"]
    combined_data = pd.DataFrame(columns=column_names)

    # Ініціалізуємо лічильники початкової кількості рядків і кінцевої
    total_initial_rows = 0
    total_final_rows = 0
    
    # Зчитаємо файли з директорії
    filenames = os.listdir(dir_for_data)

    for filename in filenames:
        # Перевіряємо, чи є файл CSV
        if not filename.endswith(".csv"):
            continue

        file_path = os.path.join(dir_for_data, filename)

        # Читання CSV файлу, пропускаємо перші 2 рядки
        df = pd.read_csv(file_path, skiprows=2, names=column_names)

        # Очищення стовпця "Year" від небажаних тегів
        df["Year"] = df["Year"].astype(str).str.replace('<tt><pre>', '').str.replace('</pre></tt>', '')
        
        # Перетворюємо стовпець Year на числовий тип
        df["Year"] = pd.to_numeric(df["Year"], errors="coerce")

        # Дістаємо Region_ID з імені файлу
        parts = filename.split('_')
        if len(parts) > 2 and parts[2].isdigit():
            region_id = int(parts[2])
        else:
            print(f"\nSkipped file: {filename}\n")
            continue  # Якщо ID не є числом, пропускаємо файл

        df["Region"] = region_id

        # Кількість початкових рядків
        initial_rows = len(df)
        total_initial_rows += initial_rows
        
        # Видаляємо записи, де у колонці VHI значення NaN
        df = df.drop(df.loc[df['VHI'] == -1].index).dropna()
        
        # Кількість рядків після очищення
        final_rows = len(df)
        total_final_rows += final_rows
        
        # Об'єднуємо дані
        combined_data = pd.concat([combined_data, df], ignore_index=True)
            
    # Обчислення відсотка видалених даних
    removed_percentage = ((total_initial_rows - total_final_rows) / total_initial_rows) * 100
    print(f"Removed {removed_percentage:.2f}% corrupted data\n")

    return combined_data

# Завантажуємо дані з директорії
combined_data = load_data_from_directory(dir_for_data)
combined_data_sorted = combined_data.sort_values(by=["Year", "Region"], ascending=[True, True])

# Зберігаємо результат у CSV файл
output_path = f"{dir_for_data}/NOAA_Ukraine.csv"
combined_data_sorted.to_csv(output_path, index=False)

print(combined_data_sorted)


Skipped file: NOAA_Ukraine.csv


Skipped file: NOAA_Ukraine_Updated.csv

Removed 2.28% corrupted data
         Year  Week    SMN     SMT    VCI    TCI    VHI Region
21860  1982.0   1.0  0.053  260.31  45.01  39.46  42.23      1
21861  1982.0   2.0  0.054  262.29  46.83  31.75  39.29      1
21862  1982.0   3.0  0.055  263.82  48.13  27.24  37.68      1
21863  1982.0   4.0  0.053  265.33  46.09  23.91  35.00      1
21864  1982.0   5.0  0.050  265.66  41.46  26.65  34.06      1
...       ...   ...    ...     ...    ...    ...    ...    ...
41529  2024.0  48.0  0.128  270.55  64.97  25.53  45.25     27
41530  2024.0  49.0  0.115  269.06  60.12  27.24  43.68     27
41531  2024.0  50.0  0.104  267.75  55.24  25.89  40.57     27
41532  2024.0  51.0  0.094  266.45  51.16  24.29  37.72     27
41533  2024.0  52.0  0.093  266.38  54.22  21.11  37.66     27

[59022 rows x 8 columns]


Реалізувати окрему процедуру, яка змінить індекси областей, які використані на порталі NOAA (за англійською абеткою) на наступні, за українською (виключно старі індекси на нові):

In [123]:
import pandas as pd
# Відповідність старих індексів (NOAA) новим українським
region_index_eng_to_ukr = {
    1: 24,  # Cherkasy -> Черкаська
    2: 26,  # Chernihiv -> Чернігівська
    3: 25,  # Chernivtsi -> Чернівецька
    4: 27,  # Crimea -> Республіка Крим
    5: 3,   # Dnipropetrovs'k -> Дніпропетровська
    6: 4,   # Donets'k -> Донецька
    7: 8,   # Ivano-Frankivs'k -> Івано-Франківська
    8: 21,  # Kharkiv -> Харківська
    9: 22,  # Kherson -> Херсонська
    10: 23, # Khmelnyts'kyi -> Хмельницька
    11: 10, # Kiev -> Київська
    12: 9,  # Kiev City -> Київ
    13: 11, # Kirovohrad -> Кіровоградська
    14: 12, # Luhans'k -> Луганська
    15: 13, # L'viv -> Львівська
    16: 14, # Mykolaiv -> Миколаївська
    17: 15, # Odessa -> Одеська
    18: 16, # Poltava -> Полтавська
    19: 17, # Rivne -> Рівненська
    20: 18, # Sevastopol' -> Севастополь
    21: 19, # Sumy -> Сумська
    22: 20, # Ternopil' -> Тернопільська
    23: 6,  # Transcarpathia (Zakarpattia) -> Закарпатська
    24: 1,  # Vinnytsia -> Вінницька
    25: 2,  # Volyn -> Волинська
    26: 7,  # Zaporizhzhia -> Запорізька
    27: 5   # Zhytomyr -> Житомирська
}

def update_region_indexes(df):
    df["Region"] = df["Region"].map(region_index_eng_to_ukr)
    return df

# Оновлення індексів областей
combined_data_updated = update_region_indexes(combined_data)
combined_data_updated_sorted = combined_data.sort_values(by=["Year", "Region"])

# Збереження оновлених даних
combined_data_updated_sorted.to_csv(f"{dir_for_data}/NOAA_Ukraine_Updated.csv", index=False)

# Вивід оновлених даних
print(combined_data_updated_sorted)

         Year  Week    SMN     SMT    VCI    TCI    VHI  Region
32790  1982.0   1.0  0.068  263.59  63.47  28.34  45.90       1
32791  1982.0   2.0  0.074  265.78  67.62  23.05  45.34       1
32792  1982.0   3.0  0.076  267.19  69.37  20.40  44.88       1
32793  1982.0   4.0  0.075  268.57  65.26  17.93  41.60       1
32794  1982.0   5.0  0.072  269.24  58.58  20.00  39.29       1
...       ...   ...    ...     ...    ...    ...    ...     ...
32785  2024.0  48.0  0.158  273.77  59.36  18.43  38.89      27
32786  2024.0  49.0  0.142  272.55  60.09  16.43  38.26      27
32787  2024.0  50.0  0.135  272.00  63.46  11.52  37.49      27
32788  2024.0  51.0  0.129  271.49  67.23   8.72  37.98      27
32789  2024.0  52.0  0.125  271.07  69.11   8.23  38.67      27

[59022 rows x 8 columns]


Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):


o Ряд VHI для області за вказаний рік;

In [130]:
def get_region_VHI_in_specified_year(data, region_id, year):
    # Фільтруємо за вказаними параметрами
    region_data = data[(data["Region"] == region_id) & (data["Year"] == year)]
    return region_data[["Year", "Week", "VHI"]]
    
region_vhi = get_region_VHI_in_specified_year(combined_data_updated_sorted, region_id=14, year=1984)
print(region_vhi)

         Year  Week    VHI
24150  1984.0   1.0  39.40
24151  1984.0   2.0  38.43
24152  1984.0   3.0  37.78
24153  1984.0   4.0  37.22
24154  1984.0   5.0  37.06
24155  1984.0   6.0  37.09
24156  1984.0   7.0  36.14
24157  1984.0   8.0  37.40
24158  1984.0   9.0  38.85
24159  1984.0  10.0  40.04
24160  1984.0  11.0  39.09
24161  1984.0  12.0  39.19
24162  1984.0  13.0  38.56
24163  1984.0  14.0  37.93
24164  1984.0  15.0  35.78
24165  1984.0  16.0  35.45
24166  1984.0  17.0  37.61
24167  1984.0  18.0  40.66
24168  1984.0  19.0  39.71
24169  1984.0  20.0  39.34
24170  1984.0  21.0  40.56
24171  1984.0  22.0  41.67
24172  1984.0  23.0  40.49
24173  1984.0  24.0  39.29
24174  1984.0  25.0  38.45
24175  1984.0  26.0  37.94
24176  1984.0  27.0  36.06
24177  1984.0  28.0  34.31
24178  1984.0  29.0  33.05
24179  1984.0  30.0  32.43
24180  1984.0  31.0  33.19
24181  1984.0  32.0  33.60
24182  1984.0  33.0  33.58
24183  1984.0  34.0  33.51
24184  1984.0  35.0  32.40
24185  1984.0  36.0  28.78
2

o Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани;

In [129]:
def find_min_max_mean_median(data, regions, years, column):
    if column not in data.columns:
        raise ValueError(f"Column '{column}' does not exist. Сhoose one of the available ones: {list(data.columns)}")

    # Фільтруємо дані за вказаними параметрами
    filtered_data = data[(data["Region"].isin(regions)) & (data["Year"].isin(years))]

    # За допомогою функції agg визначаємо екстремуми, середнє, медіану заданої колонки
    result = filtered_data.groupby(["Region", "Year"])[column].agg(
        min_value="min",
        max_value="max",
        mean_value="mean",
        median_value="median"
    ).rename(columns={
        "min_value": f"min_{column}_value",
        "max_value": f"max_{column}_value",
        "mean_value": f"mean_{column}_value",
        "median_value": f"median_{column}_value"
    }).reset_index()

    return result

try:
    extremes = find_min_max_mean_median(combined_data_updated_sorted, regions=[1, 2, 3], years=[2014, 1988], column="VHI")
    print(extremes)
except ValueError as e:
    print(e)

   Region    Year  min_VHI_value  max_VHI_value  mean_VHI_value  \
0       1  1988.0          22.58          63.04       46.365769   
1       1  2014.0          36.37          66.94       51.377115   
2       2  1988.0          34.17          66.27       52.910385   
3       2  2014.0          24.16          64.81       42.641346   
4       3  1988.0          26.95          70.20       49.354808   
5       3  2014.0          28.95          56.51       43.272308   

   median_VHI_value  
0            48.175  
1            48.200  
2            53.555  
3            40.455  
4            53.960  
5            43.195  


o Ряд VHI за вказаний діапазон років для вказаних областей;

In [131]:
def get_regions_VHI_in_specified_period(data, region_ids, start_year, end_year):
    # Фільтруємо за вказаними параметрами
    filtered_data = data[(data["Region"].isin(region_ids)) & (data["Year"] >= start_year) & (data["Year"] <= end_year)]
    return filtered_data[["Year", "Week", "Region", "VHI"]]

RegionsVHI_in_Period = get_regions_VHI_in_specified_period(combined_data_updated_sorted, region_ids=[1, 2], start_year=2014, end_year=2022)
print(RegionsVHI_in_Period)

         Year  Week  Region    VHI
34404  2014.0   1.0       1  47.80
34405  2014.0   2.0       1  50.45
34406  2014.0   3.0       1  53.22
34407  2014.0   4.0       1  52.93
34408  2014.0   5.0       1  49.86
...       ...   ...     ...    ...
50169  2022.0  48.0       2  47.74
50170  2022.0  49.0       2  46.93
50171  2022.0  50.0       2  46.14
50172  2022.0  51.0       2  47.72
50173  2022.0  52.0       2  49.82

[936 rows x 4 columns]


o Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25). Повернути роки, назви областей з екстремальними посухами та значення VHI;

In [132]:
def find_extreme_droughts(data, regions_percentage=20):
    # Визначення кількості областей
    total_regions = len(data["Region"].unique())
    regions_count = (regions_percentage / 100) * total_regions

    # Екстримальні посухи
    extreme_droughts = data[data["VHI"] <= 15]
    
    # Кількість областей з екстримальними посухами по рокам
    drought_count_by_year = extreme_droughts.groupby("Year")["Region"].nunique()

    # Вибираємо роки, де кількість областей з екстремальними посухами перевищує поріг
    extreme_years = drought_count_by_year[drought_count_by_year >=regions_count].index

    # Повертаємо відповідні дані
    result = extreme_droughts[extreme_droughts["Year"].isin(extreme_years)]
    return result[["Year", "Region", "VHI"]]

extreme_droughts = find_extreme_droughts(combined_data_updated_sorted, regions_percentage=20)
print(extreme_droughts)

         Year  Region    VHI
33740  2000.0       1  12.26
33741  2000.0       1  11.28
33742  2000.0       1  11.25
33743  2000.0       1  11.38
33744  2000.0       1  12.91
33745  2000.0       1  14.20
3135   2000.0       6  12.51
3136   2000.0       6  10.60
3137   2000.0       6  11.20
3138   2000.0       6  12.32
3139   2000.0       6  14.65
24996  2000.0      14  13.14
24997  2000.0      14   9.50
24998  2000.0      14   8.14
24999  2000.0      14   9.69
25000  2000.0      14  11.20
25001  2000.0      14  11.36
25002  2000.0      14  12.77
55601  2000.0      17  14.61
55602  2000.0      17  11.33
55603  2000.0      17   9.36
55604  2000.0      17   9.45
55605  2000.0      17   9.73
55606  2000.0      17  11.45
55607  2000.0      17  14.29
5319   2000.0      20  14.89
5320   2000.0      20  12.76
5321   2000.0      20   7.81
5322   2000.0      20   6.49
5323   2000.0      20   6.58
5324   2000.0      20   6.71
5325   2000.0      20   7.56
5326   2000.0      20   9.25
5327   2000.0 